# "Breast Cancer Prediction with Federal Learning"
### fateme mehdikhani 1403/9/2

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from imblearn.over_sampling import SMOTE

In [3]:
def load_breast_cancer_data():
    data = load_breast_cancer()
    X = data.data
    y = data.target
    return X, y

In [5]:
def preprocess_data(X, y):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y

In [7]:
def balance_data(X_train, y_train):
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_res, y_res = smote.fit_resample(X_train, y_train)
    return X_res, y_res

In [9]:
def create_model(input_dim):
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(256, activation='swish'),
        layers.Dropout(0.4),
        layers.BatchNormalization(),
        layers.Dense(128, activation='swish'),
        layers.Dropout(0.4),
        layers.BatchNormalization(),
        layers.Dense(64, activation='swish'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy', 
                  metrics=['accuracy', 'Precision', 'Recall'])
    return model

In [13]:
def main():
    X, y = load_breast_cancer_data()
    X, y = preprocess_data(X, y)


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train, y_train = balance_data(X_train, y_train)


    num_clients = 10
    client_data = np.array_split(X_train, num_clients)
    client_labels = np.array_split(y_train, num_clients)


    federated_models = []
    for i in range(num_clients):
        model = create_model(X_train.shape[1])
        model.fit(client_data[i], client_labels[i], epochs=5, batch_size=32, verbose=0)
        federated_models.append(model)


    avg_weights = [np.mean([model.get_weights()[i] for model in federated_models], axis=0) 
                   for i in range(len(federated_models[0].get_weights()))]


    for model in federated_models:
        model.set_weights(avg_weights)

 
    final_model = federated_models[0]  
    loss, accuracy, precision, recall = final_model.evaluate(X_test, y_test)
    print(f"Final Test Accuracy: {accuracy}")
    print(f"Final Test Precision: {precision}")
    print(f"Final Test Recall: {recall}")


main()


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - Precision: 0.6377 - Recall: 1.0000 - accuracy: 0.6377 - loss: 0.6919
Final Test Accuracy: 0.6228070259094238
Final Test Precision: 0.6228070259094238
Final Test Recall: 1.0
